# Project 1 : Consumer complaints resolution

## Aim : how the complaints are handelled as well as how to convincingly convey the final conlusions so build the prediction model for column "Consumer disputed"

### Author : Ganesh Patil

In [1]:
import math
import numpy as np
import pandas as pd
import time 
from sklearn.metrics import roc_auc_score 
from sklearn.model_selection import RandomizedSearchCV

In [2]:
Train_data = pd.read_csv("Consumer_Complaints_train.csv")
Test_data = pd.read_csv("Consumer_Complaints_test_share.csv")

In [3]:
Test_data['Consumer disputed?'] = pd.Series([0]* len(Test_data['Date received']))

In [4]:
Train_data['dtype'] = pd.Series(['train']*len(Train_data['Date received']))


In [5]:
Test_data['dtype'] = pd.Series(['test']*len(Test_data['Date received']))

In [6]:
Test_data = pd.DataFrame(data = Test_data)

In [7]:
Test_data.tail()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Complaint ID,Consumer disputed?,dtype
119601,2012-09-15,Credit card,NaN,Closing/Cancelling account,NaN,NaN,NaN,JPMorgan Chase & Co.,IL,60538,NaN,NaN,Web,2012-09-21,Closed with monetary relief,Yes,153482,0,test
119602,2016-07-21,Credit reporting,NaN,Credit reporting company's investigation,No notice of investigation status/result,On XXXX/XXXX/16 a dispute and validation of a ...,Company has responded to the consumer and the ...,Experian,CA,956XX,NaN,Consent provided,Web,2016-07-21,Closed with explanation,Yes,2023523,0,test
119603,2015-06-09,Debt collection,"Other (i.e. phone, health club, etc.)",Disclosure verification of debt,Not given enough info to verify debt,NaN,Company chooses not to provide a public response,"FAIR COLLECTIONS & OUTSOURCING, INC.",CA,90803,Servicemember,NaN,Phone,2015-06-12,Closed with explanation,Yes,1413678,0,test
119604,2014-02-26,Debt collection,Credit card,Disclosure verification of debt,Not given enough info to verify debt,NaN,NaN,Encore Capital Group,PA,17764,NaN,NaN,Phone,2014-02-27,Closed with explanation,Yes,732458,0,test
119605,2015-02-11,Debt collection,"Other (i.e. phone, health club, etc.)",Cont'd attempts collect debt not owed,Debt is not mine,NaN,NaN,AllianceOne Recievables Management,CA,92103,NaN,NaN,Web,2015-02-11,Closed with explanation,Yes,1235226,0,test


In [8]:
Train_data = pd.DataFrame(data = Train_data)

In [9]:
All_data = Train_data.append(Train_data)

In [10]:
Train_data.shape

(478421, 19)

In [11]:
All_data

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,dtype
0,5/15/2014,Credit card,NaN,Billing statement,NaN,NaN,NaN,Wells Fargo & Company,MI,48342,Older American,NaN,Web,5/16/2014,Closed with explanation,Yes,No,856103,train
1,9/18/2014,Bank account or service,(CD) Certificate of deposit,"Making/receiving payments, sending money",NaN,NaN,NaN,Santander Bank US,PA,18042,NaN,NaN,Referral,9/24/2014,Closed,Yes,No,1034666,train
2,3/13/2014,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,Equifax,CA,92427,NaN,NaN,Referral,4/3/2014,Closed with non-monetary relief,Yes,No,756363,train
3,7/17/2015,Credit card,NaN,Billing statement,NaN,"My credit card statement from US Bank, XXXX. X...",Company chooses not to provide a public response,U.S. Bancorp,GA,305XX,Older American,Consent provided,Web,7/17/2015,Closed with monetary relief,Yes,No,1474177,train
4,11/20/2014,Credit card,NaN,Transaction issue,NaN,NaN,NaN,Bank of America,MA,2127,NaN,NaN,Web,11/28/2014,Closed with explanation,Yes,No,1132572,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478416,6/30/2015,Bank account or service,Checking account,Problems caused by my funds being low,NaN,NaN,NaN,JPMorgan Chase & Co.,TX,77008,NaN,NaN,Phone,7/1/2015,Closed with explanation,Yes,No,1445488,train
478417,6/20/2013,Credit card,NaN,Collection practices,NaN,NaN,NaN,Citibank,AL,36111,NaN,NaN,Phone,6/21/2013,Closed with explanation,Yes,No,438011,train
478418,9/9/2014,Credit reporting,NaN,Incorrect information on credit report,Information is not mine,NaN,NaN,Equifax,TX,79201,NaN,NaN,Web,9/9/2014,Closed with explanation,Yes,No,1020651,train
478419,9/21/2012,Mortgage,Conventional fixed mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,"Select Portfolio Servicing, Inc",CA,91765,NaN,NaN,Web,9/24/2012,Closed with explanation,Yes,No,157524,train


In [12]:
All_data.isnull().sum()

Date received                        0
Product                              0
Sub-product                     276946
Issue                                0
Sub-issue                       585250
Consumer complaint narrative    806654
Company public response         776058
Company                              0
State                             7678
ZIP code                          7696
Tags                            822430
Consumer consent provided?      685868
Submitted via                        0
Date sent to company                 0
Company response to consumer         0
Timely response?                     0
Consumer disputed?                   0
Complaint ID                         0
dtype                                0
dtype: int64

In [13]:
All_data.shape

(956842, 19)

In [14]:
for col in ['Sub-product', 'Sub-issue','Consumer complaint narrative','Company public response','Tags'
           ,'Consumer consent provided?']:
    varname = col.replace('-','_').replace('?','').replace(' ','_')+ '_Nan'
    All_data[varname] = np.where(pd.isnull(All_data[col]),1,0)
    All_data.drop([col],1,inplace = True)

In [15]:
np.where(All_data['Submitted via'].isnull() == True)

(array([], dtype=int64),)

In [16]:
value = 'Web'
All_data['Submitted via'] = All_data['Submitted via'].fillna(value)

In [17]:
All_data.shape

(956842, 19)

In [18]:
All_data['Date received'] = pd.to_datetime(All_data['Date received'],infer_datetime_format=True)

In [19]:
All_data['Date sent to company'] = pd.to_datetime(All_data['Date sent to company'],infer_datetime_format=True)

In [20]:
All_data['day_diff'] = pd.to_numeric(All_data['Date sent to company']-All_data['Date received'])

In [21]:
All_data.drop(['Date received'],1, inplace = True)
All_data.drop(['Date sent to company'],1,inplace = True)

In [22]:
All_data.drop(['Company'],1, inplace = True)
All_data.drop(['ZIP code'],1,inplace = True)

In [23]:
for col in ['Product','Issue','State','Submitted via','Company response to consumer','Timely response?']:
    
    freq = All_data[col].value_counts()
    select_catgory = freq.index[freq> 200][:-1]
    
    print(col)
    
    for cat in select_catgory:
        name = col+'_'+cat
        All_data[name] = (All_data[col]== cat).astype(int)
    del All_data[col]

Product
Issue
State
Submitted via
Company response to consumer
Timely response?


In [24]:
Train_data = All_data.iloc[0:478421,]

In [25]:
Train_data.tail()

,Consumer disputed?,Complaint ID,dtype,Sub_product_Nan,Sub_issue_Nan,Consumer_complaint_narrative_Nan,Company_public_response_Nan,Tags_Nan,Consumer_consent_provided_Nan,day_diff,...,Submitted via_Referral,Submitted via_Phone,Submitted via_Postal mail,Submitted via_Fax,Company response to consumer_Closed with explanation,Company response to consumer_Closed with non-monetary relief,Company response to consumer_Closed with monetary relief,Company response to consumer_Closed without relief,Company response to consumer_Closed,Timely response?_Yes
478416,No,1445488,train,0,1,1,1,1,1,86400000000000,...,0,1,0,0,1,0,0,0,0,1
478417,No,438011,train,1,1,1,1,1,1,86400000000000,...,0,1,0,0,1,0,0,0,0,1
478418,No,1020651,train,1,0,1,1,1,1,0,...,0,0,0,0,1,0,0,0,0,1
478419,No,157524,train,0,1,1,1,1,1,259200000000000,...,0,0,0,0,1,0,0,0,0,1
478420,No,1093195,train,0,1,1,1,0,1,0,...,0,0,0,0,0,1,0,0,0,1


In [26]:
Test_data = All_data.iloc[478421:,]

In [27]:
Test_data.shape

(478421, 166)

In [28]:
Train_data['Consumer disputed?'] = np.where(Train_data['Consumer disputed?']== 'Yes',1,0)


C:\Users\ADMIN\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [29]:
x =Train_data .drop(['Complaint ID','Consumer disputed?','dtype'],1)

In [30]:
y = Train_data['Consumer disputed?']

In [31]:
Train_data = Train_data.drop(['Complaint ID','Consumer disputed?','dtype'],1)

In [32]:
from sklearn.metrics import roc_auc_score

In [33]:
from sklearn import tree

In [34]:
dtree = tree.DecisionTreeClassifier(criterion="entropy",max_leaf_nodes=10,class_weight='balanced')

In [35]:
from sklearn.ensemble import AdaBoostClassifier

In [36]:
ada = AdaBoostClassifier(base_estimator = dtree, learning_rate = 0.1,n_estimators = 500)

In [ ]:
final_model = ada.fit(x,y)

In [ ]:
Test_data1 = All_data.iloc[478421:,]

In [ ]:
Test_data1=Test_data1.drop(['Consumer disputed?','dtype'],1)

In [ ]:
prediction=np.where(final_model.predict(Test_data1.drop(['Complaint ID'],1))==1,"Yes","No")
final_prediction=pd.DataFrame(list(zip(Test_data1['Complaint ID'],list(prediction))),
                columns=['Complaint ID','Consumer disputed?'])

In [ ]:
final_prediction

In [ ]:
final_prediction.to_csv('Ganesh_patil_project_P1_Part2.csv',index=False)